In [1]:
# Importation des modules utilisés
import sqlite3
import pandas

In [2]:
# Création de la connexion
conn = sqlite3.connect("ClassicModel.sqlite")

In [3]:
# Récupération du contenu de Customers avec une requête SQL
customers = pandas.read_sql_query("SELECT * FROM Customers;", conn)
print(customers)

     customerNumber                    customerName contactLastName  \
0               103               Atelier graphique         Schmitt   
1               112              Signal Gift Stores            King   
2               114      Australian Collectors, Co.        Ferguson   
3               119               La Rochelle Gifts         Labrune   
4               121              Baane Mini Imports      Bergulfsen   
..              ...                             ...             ...   
117             486    Motor Mint Distributors Inc.       Hernandez   
118             487        Signal Collectibles Ltd.          Taylor   
119             489  Double Decker Gift Stores, Ltd           Hardy   
120             495            Diecast Collectables          Franco   
121             496               Kelly's Gift Shop         Snowden   

    contactFirstName           phone                  addressLine1  \
0             Carine      40.32.2555                54, rue Royale   
1      

# Lister les clients n’ayant jamais effecuté une commande ;

In [8]:
ex1 = pandas.read_sql_query(f"""SELECT customerName FROM Customers c 
                                LEFT JOIN Orders o 
                                ON c.customerNumber == o.customerNumber 
                                WHERE orderNumber is null;""", conn)
print(ex1)

                      customerName
0               Havel & Zbyszek Co
1           American Souvenirs Inc
2                Porto Imports Co.
3       Asian Shopping Network, Co
4                  Natürlich Autos
5                    ANG Resellers
6         Messner Shopping Network
7                Franken Gifts, Co
8                BG&E Collectables
9                 Schuyler Imports
10                Der Hund Imports
11       Cramer Spezialitäten, Ltd
12           Asian Treasures, Inc.
13            SAR Distributors, Co
14                 Kommission Auto
15          Lisboa Souveniers, Inc
16           Precious Collectables
17  Stuttgart Collectable Exchange
18        Feuer Online Stores, Inc
19                Warburg Exchange
20             Anton Designs, Ltd.
21             Mit Vergnügen & Co.
22       Kremlin Collectables, Co.
23              Raanan Stores, Inc


# Pour chaque employé, le nombre de clients, le nombre de commandes et le montant total de celles-ci

In [10]:
ex2 = pandas.read_sql_query(f"""SELECT e.employeeNumber, count(c.customerNumber) AS nb_clients,
                                        count(o.orderNumber) AS nb_commandes,
                                        sum(d.quantityOrdered * d.priceEach) AS montant
                                FROM Employees e
                                    LEFT JOIN Customers c ON e.employeeNumber == c.salesRepEmployeeNumber
                                    LEFT JOIN Orders o ON c.customerNumber == o.customerNumber
                                    LEFT JOIN OrderDetails d ON o.orderNumber == d.orderNumber
                                GROUP BY 1
                                ;""", conn)
print(ex2)

    employeeNumber  nb_clients  nb_commandes     montant
0             1002           0             0         NaN
1             1056           0             0         NaN
2             1076           0             0         NaN
3             1088           0             0         NaN
4             1102           0             0         NaN
5             1143           0             0         NaN
6             1165         331           331  1210228.57
7             1166         114           114   378064.72
8             1188         124           124   422257.44
9             1216         152           152   565516.73
10            1286         143           142   550395.19
11            1323         211           211   725598.76
12            1337         177           177   621798.01
13            1370         396           396  1401412.66
14            1401         272           272   959738.52
15            1501         236           236   808462.37
16            1504         220 

# Idem pour chaque bureau (nombre de clients, nombre de commandes et montant total), avec en plus le nombre de clients d’un pays différent, s’il y en a ;

In [47]:
ex3 = pandas.read_sql_query(f"""SELECT of.officeCode, 
                                        count(distinct c.customerNumber) AS nb_clients,
                                        count(distinct CASE WHEN of.country != c.country THEN c.customerNumber END) AS client_etr,
                                        count(o.orderNumber) AS nb_commandes,
                                        sum(d.quantityOrdered * d.priceEach) AS montant
                                FROM Offices of
                                    LEFT JOIN Employees e ON of.officeCode == e.officeCode
                                    LEFT JOIN Customers c ON e.employeeNumber == c.salesRepEmployeeNumber
                                    LEFT JOIN Orders o ON c.customerNumber == o.customerNumber
                                    LEFT JOIN OrderDetails d ON o.orderNumber == d.orderNumber
                                GROUP BY 1
                                ;""", conn)
print(ex3)

  officeCode  nb_clients  client_etr  nb_commandes     montant
0        1.0          12           0           445  1588293.29
1        2.0          12           0           276   987774.17
2        3.0          15           3           353  1275993.95
3        4.0          29          17           959  3404055.56
4        5.0           5           3           137   503957.58
5        6.0          10           5           370  1281705.83
6        7.0          17          12           456  1604168.80


# Pour chaque produit, donner le nombre de commandes, la quantité totale commandée, et le nombre de clients différents ;

In [14]:
ex4 = pandas.read_sql_query(f"""SELECT d.productCode,
                                        count(distinct o.orderNumber) AS nb_commandes,
                                        sum(d.quantityOrdered) AS quant_tot,
                                        count(distinct o.customerNumber) AS nb_clients
                                FROM OrderDetails d
                                    LEFT JOIN Orders o ON d.orderNumber == o.orderNumber
                                GROUP BY 1
                                ;""", conn)
print(ex4)

    productCode  nb_commandes  quant_tot  nb_clients
0      S10_1678            28       1026          26
1      S10_1949            28        961          27
2      S10_2016            28        999          26
3      S10_4698            28        985          25
4      S10_4757            28       1000          27
..          ...           ...        ...         ...
104   S700_3505            27        952          22
105   S700_3962            27        883          24
106   S700_4002            28       1073          26
107    S72_1253            28        960          27
108    S72_3212            27        958          24

[109 rows x 4 columns]


# Donner le nombre de commande pour chaque pays du client, ainsi que le montant total des commandes et le montant total payé : on veut conserver les clients n’ayant jamais commandé dans le résultat final ;

In [32]:
ex5 = pandas.read_sql_query(f"""
    SELECT c.country,
           COUNT(DISTINCT o.orderNumber) AS nb_commandes,
           SUM(d.total_montant_commande) AS montant_commande,
           SUM(p.amount) AS montant_paye
    FROM Customers c
    LEFT JOIN Orders o ON c.customerNumber = o.customerNumber
    LEFT JOIN (
        SELECT orderNumber, 
               SUM(quantityOrdered * priceEach) AS total_montant_commande
        FROM OrderDetails
        GROUP BY orderNumber
    ) d ON o.orderNumber = d.orderNumber
    LEFT JOIN Payments p ON c.customerNumber = p.customerNumber
    GROUP BY c.country;
    """, conn)
print(ex5)

         country  nb_commandes  montant_commande  montant_paye
0      Australia            19        2182269.38    2352253.84
1        Austria             7         606187.59     487494.02
2        Belgium             7         283705.44     333442.11
3         Canada             7         448157.12     421283.00
4        Denmark             7         781357.50     796334.75
5        Finland             9         988745.73     988745.73
6         France            37        3160296.75    3324693.52
7        Germany             7         576293.44     576293.44
8      Hong Kong             2          48784.36      97568.72
9        Ireland             2         115512.86     115512.86
10        Israel             0               NaN           NaN
11         Italy            10         945208.16     963279.67
12         Japan             6         496898.36     570898.86
13   Netherlands             0               NaN           NaN
14   New Zealand            15        1736137.04    166

# On veut la table de contigence du nombre de commande entre la ligne de produits et le pays du client ;

In [38]:
ex6 = pandas.read_sql_query(f"""
    SELECT c.country, 
           p.productLine, 
           COUNT(DISTINCT o.orderNumber) AS nb_commandes
    FROM Customers c
    JOIN Orders o ON c.customerNumber = o.customerNumber
    JOIN OrderDetails d ON o.orderNumber = d.orderNumber
    JOIN Products p ON d.productCode = p.productCode
    GROUP BY c.country, p.productLine
    ORDER BY c.country, p.productLine;
    """, conn)

print(ex6)

       country       productLine  nb_commandes
0    Australia      Classic Cars            12
1    Australia       Motorcycles             6
2    Australia            Planes             5
3    Australia             Ships             2
4    Australia            Trains             1
..         ...               ...           ...
121        USA            Planes            22
122        USA             Ships            25
123        USA            Trains            15
124        USA  Trucks and Buses            27
125        USA      Vintage Cars            67

[126 rows x 3 columns]


In [40]:
# Créer la table de contingence avec pivot_table
contingency_table = ex6.pivot_table(index='productLine', columns='country', values='nb_commandes', fill_value=0)

# Afficher le résultat
print(contingency_table)

country           Australia  Austria  Belgium  Canada  Denmark  Finland  \
productLine                                                               
Classic Cars           12.0      5.0      2.0     6.0      5.0      9.0   
Motorcycles             6.0      1.0      0.0     1.0      0.0      2.0   
Planes                  5.0      2.0      1.0     2.0      1.0      2.0   
Ships                   2.0      1.0      3.0     2.0      3.0      2.0   
Trains                  1.0      0.0      3.0     0.0      2.0      1.0   
Trucks and Buses        5.0      1.0      0.0     3.0      1.0      2.0   
Vintage Cars           14.0      4.0      5.0     5.0      3.0      2.0   

country           France  Germany  Hong Kong  Ireland  ...  Japan  \
productLine                                            ...          
Classic Cars        23.0      4.0        0.0      1.0  ...    3.0   
Motorcycles         15.0      2.0        1.0      1.0  ...    2.0   
Planes               4.0      1.0        2.0    

# On veut la même table croisant la ligne de produits et le pays du client, mais avec le montant total payé dans chaque cellule ;

In [48]:
ex7 = pandas.read_sql_query(f"""
    SELECT c.country, 
           p.productLine, 
           sum(DISTINCT pa.amount) AS montant
    FROM Customers c
    JOIN Orders o ON c.customerNumber = o.customerNumber
    JOIN OrderDetails d ON o.orderNumber = d.orderNumber
    JOIN Products p ON d.productCode = p.productCode
    JOIN Payments pa ON c.customerNumber = pa.customerNumber
    GROUP BY c.country, p.productLine
    ORDER BY c.country, p.productLine;
    """, conn)

print(ex7)

       country       productLine     montant
0    Australia      Classic Cars   566642.90
1    Australia       Motorcycles   516429.53
2    Australia            Planes   412646.77
3    Australia             Ships   157812.92
4    Australia            Trains    59469.12
..         ...               ...         ...
121        USA            Planes  1524097.51
122        USA             Ships  1933780.17
123        USA            Trains  1653147.49
124        USA  Trucks and Buses  2306979.48
125        USA      Vintage Cars  3162135.35

[126 rows x 3 columns]


In [49]:
# Créer la table de contingence avec pivot_table
contingency_table2 = ex7.pivot_table(index='productLine', columns='country', values='montant', fill_value=0)

# Afficher le résultat
print(contingency_table2)

country           Australia    Austria    Belgium     Canada    Denmark  \
productLine                                                               
Classic Cars      566642.90  134939.48  100000.67  190385.59  219624.28   
Motorcycles       516429.53   82675.58       0.00   74634.85       0.00   
Planes            412646.77   82675.58   33440.10   74634.85  119028.73   
Ships             157812.92   82675.58  100000.67  115146.67  219624.28   
Trains             59469.12       0.00  100000.67       0.00  219624.28   
Trucks and Buses  516429.53   52263.90       0.00  115750.74  100595.55   
Vintage Cars      566642.90  134939.48  100000.67  190385.59  219624.28   

country             Finland      France    Germany  Hong Kong   Ireland  ...  \
productLine                                                              ...   
Classic Cars      329581.91  1017384.70  185478.17       0.00  57756.43  ...   
Motorcycles       225211.53   694429.58  100306.58   48784.36  57756.43  ...   
Plan

# Donner les 10 produits pour lesquels la marge moyenne est la plus importante (cf buyPrice et priceEach) ;

In [53]:
ex8 = pandas.read_sql_query(f"""
    SELECT distinct p.productCode, p.productName, 
        AVG(d.priceEach - p.buyPrice) AS marge
    FROM Products p
        JOIN OrderDetails d ON p.productCode = d.productCode
    GROUP BY 1, 2
    ORDER BY 3 DESC
    LIMIT 10;
    """, conn)

print(ex8)

  productCode                           productName      marge
0    S10_1949              1952 Alpine Renault 1300  99.006429
1    S10_4698  2003 Harley-Davidson Eagle Drag Bike  95.235000
2    S18_3232           1992 Ferrari 360 Spider red  83.334906
3    S12_2823                      2002 Suzuki XREO  83.201429
4    S18_2795                1928 Mercedes-Benz SSK  82.696786
5    S12_1108                     2001 Ferrari Enzo  81.043704
6    S12_3891                      1969 Ford Falcon  77.335926
7    S18_3685        1948 Porsche Type 356 Roadster  72.636800
8    S18_2870          1999 Indy 500 Monte Carlo SS  71.794400
9    S18_1749              1917 Grand Touring Sedan  70.432800


# Lister les produits (avec le nom et le code du client) qui ont été vendus à perte

In [60]:
ex9 = pandas.read_sql_query(f"""
    SELECT p.productName, c.customerNumber, c.customerName,
            p.buyPrice, d.priceEach
    FROM Products p
        JOIN OrderDetails d ON p.productCode = d.productCode
        JOIN Orders o ON d.orderNumber = o.orderNumber
        JOIN Customers c ON c.customerNumber = o.customerNumber
    WHERE p.buyPrice > d.priceEach
    ORDER BY buyPrice desc
    """, conn)

print(ex9)

                              productName  customerNumber  \
0                  1962 LanciaA Delta 16V             363   
1                  1962 LanciaA Delta 16V             276   
2                  1962 LanciaA Delta 16V             276   
3          1998 Chrysler Plymouth Prowler             124   
4          1998 Chrysler Plymouth Prowler             323   
..                                    ...             ...   
74                 1999 Yamaha Speed Boat             145   
75              1969 Chevrolet Camaro Z28             187   
76  1969 Harley Davidson Ultimate Chopper             119   
77                     1949 Jaguar XK 120             141   
78                     1934 Ford V8 Coupe             363   

                    customerName  buyPrice  priceEach  
0   Online Diecast Creations Co.    103.42      61.99  
1        Anna's Decorations, Ltd    103.42      65.63  
2        Anna's Decorations, Ltd    103.42      46.90  
3   Mini Gifts Distributors Ltd.    101.51 

# (bonus) Lister les clients pour lesquels le montant total payé est inférieur aux montants totals des achats

In [75]:
ex10 = pandas.read_sql_query(f"""
    SELECT c.customerName,
        sum(p.amount) AS montant_paye,
        sum(d.priceEach * d.quantityOrdered) AS montant_achat
    FROM Customers c
        JOIN Payments p ON c.customerNumber = p.customerNumber
        JOIN Orders o ON c.customerNumber = o.customerNumber
        JOIN OrderDetails d ON o.orderNumber = d.orderNumber
    GROUP BY 1
    HAVING montant_paye > montant_achat
;
    """, conn)

print(ex10)

                   customerName  montant_paye  montant_achat
0                AV Stores, Co.    8048198.31      473423.43
1                  Alpha Cognac    1409768.80      211465.32
2            Amica Models & Co.    2447048.76      188234.52
3       Anna's Decorations, Ltd    7083821.98      615984.52
4             Atelier graphique     169259.72       72539.88
..                          ...           ...            ...
93              Vida Sport, Ltd    3649120.36      235427.12
94              Vitachrome Inc.    2201031.50      264123.78
95     Volvo Model Replicas, Co    1439342.72      151509.76
96  West Coast Collectables Co.     599100.32       92169.28
97            giftsbymail.co.uk    2034261.84      156481.68

[98 rows x 3 columns]


In [74]:
ex10 = pandas.read_sql_query(f"""
    SELECT c.customerName,
       SUM(p.amount) AS montant_paye,
       SUM(d.priceEach * d.quantityOrdered) AS montant_achat
FROM Customers c
JOIN Payments p ON c.customerNumber = p.customerNumber
JOIN Orders o ON c.customerNumber = o.customerNumber
JOIN OrderDetails d ON o.orderNumber = d.orderNumber
GROUP BY c.customerName
HAVING montant_paye < montant_achat;

    """, conn)

print(ex10)

Empty DataFrame
Columns: [customerName, montant_paye, montant_achat]
Index: []
